In [9]:
from anthropic import AnthropicVertex

LOCATION="asia-southeast1" # or "us-central1"

client = AnthropicVertex(region=LOCATION, project_id="website-254017")

message = client.messages.create(
  max_tokens=4096,
  messages=[
    {
      "role": "user",
      "content": "Send me a recipe for banana bread.",
    }
  ],
  model="claude-3-sonnet@20240229",
)
print(message.model_dump_json(indent=2))

{
  "id": "msg_vrtx_016ZhEahK5tBz4tfTybcPbgG",
  "content": [
    {
      "text": "Here is a classic recipe for delicious banana bread:\n\nBanana Bread\n\nIngredients:\n- 1 3/4 cups (220g) all-purpose flour\n- 1 teaspoon baking soda\n- 1/4 teaspoon salt\n- 1/2 cup (115g) unsalted butter, softened\n- 3/4 cup (150g) granulated sugar\n- 2 large eggs\n- 1 teaspoon vanilla extract\n- 1 1/4 cups (300g) mashed ripe bananas (about 3 medium bananas)\n\nInstructions:\n\n1. Preheat oven to 350°F (177°C). Grease a 9x5 inch loaf pan and set aside.\n\n2. In a medium bowl, whisk together the flour, baking soda and salt. Set aside.\n\n3. In a large bowl, beat the butter and sugar together until light and fluffy, about 2-3 minutes. Beat in the eggs one at a time, then stir in the vanilla and mashed bananas until combined. \n\n4. Fold the dry ingredients into the wet ingredients until just combined, being careful not to overmix.\n\n5. Pour the batter into the prepared loaf pan and bake for 55-65 minutes

In [10]:
message.content[0].text

'Here is a classic recipe for delicious banana bread:\n\nBanana Bread\n\nIngredients:\n- 1 3/4 cups (220g) all-purpose flour\n- 1 teaspoon baking soda\n- 1/4 teaspoon salt\n- 1/2 cup (115g) unsalted butter, softened\n- 3/4 cup (150g) granulated sugar\n- 2 large eggs\n- 1 teaspoon vanilla extract\n- 1 1/4 cups (300g) mashed ripe bananas (about 3 medium bananas)\n\nInstructions:\n\n1. Preheat oven to 350°F (177°C). Grease a 9x5 inch loaf pan and set aside.\n\n2. In a medium bowl, whisk together the flour, baking soda and salt. Set aside.\n\n3. In a large bowl, beat the butter and sugar together until light and fluffy, about 2-3 minutes. Beat in the eggs one at a time, then stir in the vanilla and mashed bananas until combined. \n\n4. Fold the dry ingredients into the wet ingredients until just combined, being careful not to overmix.\n\n5. Pour the batter into the prepared loaf pan and bake for 55-65 minutes, until a toothpick inserted in the center comes out clean.\n\n6. Allow the bread 

In [11]:
from anthropic import AnthropicVertex

LOCATION="europe-west1" # or "us-east5"

client = AnthropicVertex(region=LOCATION, project_id="website-254017")

message = client.messages.create(
  max_tokens=4096,
  messages=[
    {
      "role": "user",
      "content": "Send me a recipe for banana bread.",
    }
  ],
  model="claude-3-5-sonnet@20240620",
)
print(message.model_dump_json(indent=2))

{
  "id": "msg_vrtx_01AR6jTeApfNJMmjXUewK1L6",
  "content": [
    {
      "text": "Here's a simple recipe for delicious banana bread:\n\nIngredients:\n- 2-3 ripe bananas, mashed\n- 1/3 cup melted butter\n- 1/2 cup sugar\n- 1 egg\n- 1 teaspoon vanilla extract\n- 1 teaspoon baking soda\n- Pinch of salt\n- 1 1/2 cups all-purpose flour\n- Optional: 1/2 cup chopped nuts or chocolate chips\n\nInstructions:\n\n1. Preheat oven to 350°F (175°C). Grease a 4x8-inch loaf pan.\n\n2. In a large bowl, mix mashed bananas with melted butter.\n\n3. Stir in sugar, egg, and vanilla extract.\n\n4. Sprinkle baking soda and salt over the mixture and stir in.\n\n5. Add flour and mix until just combined. Don't overmix.\n\n6. Fold in nuts or chocolate chips if using.\n\n7. Pour batter into the prepared loaf pan.\n\n8. Bake for 50-60 minutes, or until a toothpick inserted into the center comes out clean.\n\n9. Let cool in the pan for a few minutes, then remove and cool completely on a wire rack.\n\nEnjoy your ho

In [31]:
from typing import Optional, List, Mapping, Any

from llama_index.core import SimpleDirectoryReader, SummaryIndex
from llama_index.core.callbacks import CallbackManager
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core import Settings
from anthropic.lib.vertex._client import AnthropicVertex



class OurLLM(CustomLLM):
    context_window: int = 4096
    num_output: int = 256
    model_name: str = "custom"
    client: Optional[AnthropicVertex] = None

    def __init__(self):
        super().__init__()
        self.client = None  # Initialize as None, will be created when needed

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model_name,
        )

    def _ensure_client(self):
        if self.client is None:
            self.client = AnthropicVertex(region="europe-west1", project_id="website-254017")

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        self._ensure_client()
        message = self.client.messages.create(
            max_tokens=4096,
            messages=[
                {
                "role": "user",
                "content": prompt,
                }
            ],
            model="claude-3-5-sonnet@20240620",
        )
        return CompletionResponse(text=message.content[0].text)

    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> CompletionResponseGen:
        self._ensure_client()
        response = ""
        with self.client.messages.stream(
            max_tokens=4096,
            messages=[
                {
                "role": "user",
                "content": prompt,
                }
            ],
            model="claude-3-5-sonnet@20240620",
        ) as stream:
            for text in stream.text_stream:
                response += text
                yield CompletionResponse(text=response, delta=text)

# define our LLM
llm = OurLLM()

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region="europe-west1", project_id="website-254017")

with client.messages.stream(
  max_tokens=1024,
  messages=[
    {
      "role": "user",
      "content": "Send me a recipe for banana bread.",
    }
  ],
  model="claude-3-5-sonnet@20240620",
) as stream:
  for text in stream.text_stream:
    print(text)

Here's
 a simple
 recipe for del
icious banana bread:


Ingredients:

- 2
-
3 ripe
 bananas, m
ashed

- 1/
3
 cup mel
ted butter

- 1/
2
 cup sugar

- 1
 egg
-
 1 te
aspoon vanilla extract

- 1
 te
aspoon baking
 soda
-
 1/4
 teaspoon
 salt
- 
1 1/
2 cups all
-purpose flour

- Optional
: 
1/2
 cup choppe
d nuts
 or
 chocolate chips

Instructions
:

1. P
reheat 
oven to 350
°F (175
°C).
 Grease a
 4
x
8-
inch loaf pan
.


2. In a
 large bowl, mix
 m
ashed bananas with
 mel
ted butter.


3.
 Stir in sugar
, egg, an
d vanilla extract
.


4.
 Spr
inkle baking
 soda an
d salt over the mixture
 an
d st
ir in
.

5.
 Ad
d flour and mix
 until just combined.
 Don
't overmix
.

6.
 Fold in optional
 nuts or chocolate chips
 if
 using.

7
. Pour batter
 into prepared loaf
 pan.

8
. Bake
 for 50
-60 minutes
, or
 until a tooth
pick inserted into
 the center comes out
 clean.

9
. Let
 cool in the pan
 for a
 few minutes, then
 remove
 and cool
 completely on a wire
 rack.


Enjoy your hom
emade b

In [33]:
llm.stream_complete("Hi")

<generator object llm_completion_callback.<locals>.wrap.<locals>.wrapped_llm_predict.<locals>.wrapped_gen at 0x17ac7e960>

In [30]:
from anthropic.lib.vertex._client import AnthropicVertex

In [29]:
type(client)

anthropic.lib.vertex._client.AnthropicVertex

In [17]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region="europe-west1", project_id="website-254017")

with client.messages.stream(
  max_tokens=1024,
  messages=[
    {
      "role": "user",
      "content": "Send me a recipe for banana bread.",
    }
  ],
  model="claude-3-5-sonnet@20240620",
) as stream:
  for text in stream.text_stream:
    print(text)

Here's
 a simple
 recipe for del
icious banana bread:


Ingredients:

- 2
-
3 ripe
 bananas, m
ashed

- 1/
3
 cup mel
ted butter

- 1/
2
 cup sugar

- 1
 egg
-
 1 te
aspoon vanilla extract

- 1
 te
aspoon baking
 soda
-
 1/4
 teaspoon
 salt
- 
1 1/
2 cups all
-purpose flour

- Optional
: 
1/2
 cup choppe
d nuts
 or
 chocolate chips

Instructions
:

1. P
reheat 
oven to 350
°F (175
°C).
 Grease a
 4
x
8-
inch loaf pan
.


2. In a
 large bowl, mix
 m
ashed bananas with
 mel
ted butter.


3.
 Stir in sugar
, egg, an
d vanilla extract
.


4.
 Spr
inkle baking
 soda an
d salt over the mixture
 an
d st
ir in
.

5.
 Ad
d flour and mix
 until just combined.
 Don
't overmix
.

6.
 Fold in optional
 nuts or chocolate chips
 if
 using.

7
. Pour batter
 into prepared loaf
 pan.

8
. Bake
 for 50
-60 minutes
, or
 until a tooth
pick inserted into
 the center comes out
 clean.

9
. Let
 cool in the pan
 for a
 few minutes, then
 remove
 and cool
 completely on a wire
 rack.


Enjoy your hom
emade b

In [24]:
client